In [14]:
import pandas as pd
from scapy.all import rdpcap
import plotly.graph_objects as go
import ipython

In [9]:
PCAP_FOLDER = "/home/sebastfu/tcpreplay/effect_tcpreplay"
NR = 1

In [3]:
def pcap_to_dataframe(pcap_file):
    """
    Function to read packets from a pcap file and convert them into a DataFrame.
    """
    packets = rdpcap(pcap_file)
    
    # Initialize lists to store data
    timestamps = []
    src_ips = []
    dst_ips = []
    
    # Extract timestamp, source IP, and destination IP from each packet
    for packet in packets:
        timestamps.append(packet.time)
        src_ips.append(packet[1].src)  # Assuming IP is at layer 1 (index 1)
        dst_ips.append(packet[1].dst)  # Assuming IP is at layer 1 (index 1)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Timestamp': timestamps,
        'SrcIP': src_ips,
        'DstIP': dst_ips
    })

    df['iat'] = df['Timestamp'].diff()
    
    return df

In [12]:
def getDurationOfPcap(df):
    return (df["Timestamp"].max() - df["Timestamp"].min())

def getAmountOfPackets(df):
    return len(df.index)


In [10]:
originalDF = pcap_to_dataframe(f"{PCAP_FOLDER}/replay_{NR}.pcapng")
replayDF= pcap_to_dataframe(f"{PCAP_FOLDER}/tcpreplay_validation_capture_{NR}.pcapng")

In [13]:
durationOriginal = getDurationOfPcap(originalDF)
durationReplay = getDurationOfPcap(replayDF)

print(f"Duration of original pcap is {durationOriginal}, and duration of replay is {durationReplay}")

Duration of original pcap is 99.983246, and duration of replay is 99.983220197


In [16]:
# Assuming you have already created the DataFrames df1 and df2 with the 'TimeDiff' column
fig = go.Figure()

# Add line plot for df1
fig.add_trace(go.Scatter(x=originalDF.index, y=originalDF['iat'], mode='lines', name='original DF'))

# Add line plot for df2
fig.add_trace(go.Scatter(x=replayDF.index, y=replayDF['iat'], mode='lines', name='replay DF'))

# Update layout
fig.update_layout(title='Inter-Arrival Time Line Plot',
                  xaxis_title='Packet Index',
                  yaxis_title='Inter-Arrival Time')

# Show plot
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed